<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/SECOP_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""SECOP_API - Ajustado para columnas reales"""

# SECOP II

# 1. Instalaciones
# Solo ejecutar si no tienes las librerías instaladas
!pip install pandas requests

import pandas as pd
import requests
from datetime import datetime
import time

def buscar_contratos(limite=100, **filtros):
    """
    Busca contratos en SECOP II con filtros personalizables
    """
    url = "https://www.datos.gov.co/resource/rpmr-utcd.json"

    # Construir parámetros de consulta
    params = {"$limit": limite}

    # Mapeo de nombres de parámetros (ajustado a las columnas reales del SECOP)
    mapeo_campos = {
        'nivel_entidad': 'nivel_entidad',
        'codigo_entidad': 'codigo_entidad_en_secop',
        'nombre_entidad': 'nombre_de_la_entidad',
        'nit_entidad': 'nit_de_la_entidad',
        'departamento': 'departamento_entidad',
        'municipio': 'municipio_entidad',
        'estado_proceso': 'estado_del_proceso',
        'modalidad': 'modalidad_de_contrataci_n',
        'objeto_a_contratar': 'objeto_a_contratar',
        'objeto_proceso': 'objeto_del_proceso',
        'numero_proceso': 'numero_de_proceso',
        'valor_contrato': 'valor_contrato',
        'nombre_contratista': 'nom_raz_social_contratista',
        'url_contrato': 'url_contrato',
        'origen': 'origen',
        'tipo_documento': 'tipo_documento_proveedor',
        'documento_proveedor': 'documento_proveedor',
        'fecha_firma': 'fecha_de_firma_del_contrato',
        'fecha_inicio': 'fecha_inicio_ejecuci_n',
        'fecha_fin': 'fecha_fin_ejecuci_n',
    }

    # Aplicar filtros simples
    for filtro_key, filtro_value in filtros.items():
        if filtro_value and filtro_key in mapeo_campos:
            campo_api = mapeo_campos[filtro_key]
            if 'objeto' in filtro_key:
                params[campo_api] = {'$like': f'%{filtro_value}%'}
            else:
                params[campo_api] = filtro_value

    # Construir filtros WHERE para rangos
    where_conditions = []

    if filtros.get('valor_minimo', 0) > 0:
        where_conditions.append(f"valor_contrato >= {filtros['valor_minimo']}")

    if filtros.get('valor_maximo', 0) > 0:
        where_conditions.append(f"valor_contrato <= {filtros['valor_maximo']}")

    if filtros.get('fecha_firma_desde'):
        where_conditions.append(f"fecha_de_firma_del_contrato >= '{filtros['fecha_firma_desde']}'")

    if filtros.get('fecha_firma_hasta'):
        where_conditions.append(f"fecha_de_firma_del_contrato <= '{filtros['fecha_firma_hasta']}'")

    if filtros.get('fecha_inicio_desde'):
        where_conditions.append(f"fecha_inicio_ejecuci_n >= '{filtros['fecha_inicio_desde']}'")

    if filtros.get('fecha_inicio_hasta'):
        where_conditions.append(f"fecha_inicio_ejecuci_n <= '{filtros['fecha_inicio_hasta']}'")

    if where_conditions:
        params['$where'] = ' AND '.join(where_conditions)

    # Realizar petición con reintentos
    max_intentos = 3
    for intento in range(max_intentos):
        try:
            response = requests.get(url, params=params, timeout=30)

            if response.status_code == 200:
                data = response.json()
                return pd.DataFrame(data) if data else pd.DataFrame()

            elif response.status_code == 429:  # Rate limit
                if intento < max_intentos - 1:
                    time.sleep(2 ** intento)
                    continue

            print(f"Error HTTP: {response.status_code}")
            return pd.DataFrame()

        except Exception as e:
            if intento < max_intentos - 1:
                time.sleep(2)
                continue
            print(f"Error: {e}")
            return pd.DataFrame()

    return pd.DataFrame()

def mostrar_columnas_disponibles():
    """Muestra las columnas disponibles en SECOP II"""
    df_muestra = buscar_contratos(limite=1)
    if not df_muestra.empty:
        print("Columnas disponibles en SECOP II:")
        for i, col in enumerate(df_muestra.columns, 1):
            print(f"{i:2d}. {col}")
    return df_muestra.columns.tolist() if not df_muestra.empty else []

def exportar_excel(df, prefijo="secop"):
    """Exporta DataFrame a Excel con timestamp"""
    if df.empty:
        print("Sin datos para exportar")
        return None

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    archivo = f"{prefijo}_{timestamp}.xlsx"

    try:
        df.to_excel(archivo, index=False)
        return archivo
    except Exception as e:
        print(f"Error al exportar: {e}")
        return None

def resumen_busqueda(df):
    """Muestra resumen de resultados"""
    if df.empty:
        print("No se encontraron contratos")
        return

    print(f"Encontrados: {len(df)} contratos")

    # Mostrar primeras 10 columnas
    cols_mostrar = list(df.columns)[:10]
    if len(df.columns) > 10:
        cols_mostrar.append(f"... y {len(df.columns) - 10} más")
    print(f"Columnas: {cols_mostrar}")

    # Verificar objeto del contrato
    if 'objeto_a_contratar' in df.columns:
        objetos_validos = df['objeto_a_contratar'].notna().sum()
        objetos_definidos = df[df['objeto_a_contratar'] != 'NO DEFINIDO']['objeto_a_contratar'].notna().sum()
        print(f"Contratos con objeto definido: {objetos_definidos}/{len(df)} ({objetos_validos} válidos)")



In [2]:
# ==============================================================================
# BLOQUE DE EJECUCIÓN - MODIFICA AQUÍ LOS FILTROS
# ==============================================================================

# Variables de filtros
limite = 10
nivel_entidad = ""
codigo_entidad = ""
nombre_entidad = ""
nit_entidad = ""
departamento = "La Guajira"
municipio = ""
estado_proceso = ""
modalidad = ""
objeto_a_contratar = ""
objeto_proceso = ""
numero_proceso = ""
valor_contrato = ""
nombre_contratista = ""
url_contrato = ""
origen = ""
tipo_documento = ""
documento_proveedor = ""
valor_minimo = 0
valor_maximo = 0
fecha_firma_desde = ""
fecha_firma_hasta = ""
fecha_inicio_desde = ""
fecha_inicio_hasta = ""

# Ejecutar búsqueda
df = buscar_contratos(
    limite=limite,
    nivel_entidad=nivel_entidad,
    codigo_entidad=codigo_entidad,
    nombre_entidad=nombre_entidad,
    nit_entidad=nit_entidad,
    departamento=departamento,
    municipio=municipio,
    estado_proceso=estado_proceso,
    modalidad=modalidad,
    objeto_a_contratar=objeto_a_contratar,
    objeto_proceso=objeto_proceso,
    numero_proceso=numero_proceso,
    valor_contrato=valor_contrato,
    nombre_contratista=nombre_contratista,
    url_contrato=url_contrato,
    origen=origen,
    tipo_documento=tipo_documento,
    documento_proveedor=documento_proveedor,
    valor_minimo=valor_minimo,
    valor_maximo=valor_maximo,
    fecha_firma_desde=fecha_firma_desde,
    fecha_firma_hasta=fecha_firma_hasta,
    fecha_inicio_desde=fecha_inicio_desde,
    fecha_inicio_hasta=fecha_inicio_hasta
)

# Guardar resultados
if not df.empty:
    archivo = exportar_excel(df)
    if archivo:
        print(f"Guardado en: {archivo}")
    resumen_busqueda(df)
else:
    print("No se encontraron contratos")

# Análisis rápido
df


Guardado en: secop_20250722_134556.xlsx
Encontrados: 10 contratos
Columnas: ['nivel_entidad', 'codigo_entidad_en_secop', 'nombre_de_la_entidad', 'nit_de_la_entidad', 'departamento_entidad', 'municipio_entidad', 'estado_del_proceso', 'modalidad_de_contrataci_n', 'objeto_a_contratar', 'objeto_del_proceso', '... y 12 más']
Contratos con objeto definido: 10/10 (10 válidos)


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,...,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor
0,TERRITORIAL,244847027,LA GUAJIRA - INSTITUCIÓN ETNOEDUCATIVA INTEGRA...,901167840,La Guajira,Uribia,Liquidado,Régimen Especial,COMPRA DE POLIZAS DE GARANTIA DE BUEN MANEJO D...,COMPRA DE PoLIZAS DE GARANTiA DE BUEN MANEJO D...,...,2020-01-17T00:00:00.000,2020-01-27T00:00:00.000,20-4-10732881,2020-001,714000,LA PREVISORA S.A,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Nit de Persona Jurídica,860002400
1,Nacional,703458380,ICBF REGIONAL GUAJIRA,899999239,La Guajira,No Definido,Modificado,Contratación directa,PRESTAR SERVICIOS PROFESIONALES PARA LA DIRECC...,PRESTAR SERVICIOS PROFESIONALES PARA LA DIRECC...,...,2021-01-26T00:00:00.000,2021-12-31T00:00:00.000,CO1.PCCNTR.2169033,44000872021,42388667,SHIRLY JULIETH RADILLO GUARDIOLA,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1118861807
2,Territorial,702424037,ALCALDIA DISTRITO DE RIOHACHA,892115007,La Guajira,Riohacha,Modificado,Contratación directa,PRESTACION DE \nSERVICIOS ASISTENCIALES DE APO...,PRESTACION DE \nSERVICIOS ASISTENCIALES DE APO...,...,2022-02-03T00:00:00.000,2022-12-09T00:00:00.000,CO1.PCCNTR.3557203,CPS-253-2022,22669064,ELSI ENITH PADILLA CANTILLO,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,40936837
3,NACIONAL,122039000,INSTITUTO NACIONAL DE FORMACIÓN TÉCNICA PROFES...,860402193,La Guajira,San Juan del Cesar,Celebrado,Contratación Directa (Ley 1150 de 2007),PRESTA SERVICIOS PROFESIONALES JURIDICOS BRIND...,PRESTA SERVICIOS PROFESIONALES JURIDICOS BRIND...,...,2025-06-24T00:00:00.000,2025-12-06T00:00:00.000,25-12-14444892,CPS-141-2025,13337500,ELIANA ISABEL GONZALEZ SUAREZ,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Cédula de Ciudadanía,1122416173
4,Nacional,704156207,SENA REGIONAL GUAJIRA Grupo Mixto de Apoyo Adm...,899999034,La Guajira,Riohacha,En ejecución,Mínima cuantía,ADQUIRIR A TITULO DE COMPRAVENTA MATERIALES EQ...,ADQUIRIR A TITULO DE COMPRAVENTA MATERIALES EQ...,...,2024-04-01T00:00:00.000,2024-04-22T00:00:00.000,CO1.PCCNTR.6140413,CO1.PCCNTR.6140413,116221270,JP SOLUCIONES COLOMBIA SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901247893
5,Territorial,702424037,ALCALDIA DISTRITO DE RIOHACHA,892115007,La Guajira,Riohacha,En ejecución,Contratación directa,PRESTACIÓN DE SERVICIOS DE APOYO A LA GESTIÓN ...,PRESTACIÓN DE SERVICIOS DE APOYO A LA GESTIÓN ...,...,2024-04-04T00:00:00.000,2024-12-27T00:00:00.000,CO1.PCCNTR.6170532,CPS-097-2024,22083333,SULIS CENIT COTES LINDO,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,40924957
6,Territorial,700098031,LA GUAJIRA - ALCALDÍA MUNICIPIO DE MAICAO,892120020,La Guajira,Maicao,En ejecución,Contratación directa,PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...,PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...,...,2024-04-30T00:00:00.000,2024-11-29T00:00:00.000,CO1.PCCNTR.6253474,CPS-CD-MM-328-2024,13650000,MELQUICEDEC SARDOT SANCHEZ,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1124023854
7,TERRITORIAL,244000063,LA GUAJIRA - E.S.E. HOSPITAL SAN JOSÉ DE MAICAO,892120115,La Guajira,Maicao,Celebrado,Régimen Especial,PRESTACION DE SERVICIOS DE APOYO TECNICO ADMIN...,PRESTACIoN DE SERVICIOS DE APOYO TeCNICO ADMIN...,...,2019-01-01T00:00:00.000,2019-04-01T00:00:00.000,19-4-9144225,374-19,18000000,YUDITH QUINTERO RAMIREZ,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Cédula de Ciudadanía,60398430
8,Territorial,702594383,GOBERNACIÓN DE LA GUAJIRA,8921150151,La Guajira,Riohacha,Modificado,Contratación directa,PRESTACIÓN DE SERVICIOS PROFESIONALES

In [3]:
#Analizar DataSet
print(df.loc[0, 'objeto_a_contratar'])
print(df.loc[0, 'url_contrato'])

print(df.iloc[0:5][['nombre_de_la_entidad', 'departamento_entidad']])

COMPRA DE POLIZAS DE GARANTIA DE BUEN MANEJO DE RECURSOS PUBLICOS QUE CUBRAN Y AMPAREN A LA RECTORA Y AL PAGADOR DE LA INSTITUCION ETNOEDUCATIVA INTEGRAL RURAL INDIGENA DE MALEIWAMANA MUNICIPIO DE URIBIA  LA GUAJIRA
https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=20-4-10732881
                                nombre_de_la_entidad departamento_entidad
0  LA GUAJIRA - INSTITUCIÓN ETNOEDUCATIVA INTEGRA...           La Guajira
1                              ICBF REGIONAL GUAJIRA           La Guajira
2                      ALCALDIA DISTRITO DE RIOHACHA           La Guajira
3  INSTITUTO NACIONAL DE FORMACIÓN TÉCNICA PROFES...           La Guajira
4  SENA REGIONAL GUAJIRA Grupo Mixto de Apoyo Adm...           La Guajira
